In [10]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline
kernel_size=3
low_threshold = 50
high_threshold = 150

threshold = 5    # minimum number of votes (intersections in Hough grid cell)


# Hough Transform
rho = 1 # distance resolution in pixels of the Hough grid
theta = 1 * np.pi/180 # angular resolution in radians of the Hough grid
threshold = 15     # minimum number of votes (intersections in Hough grid cell)
min_line_length = 10 #minimum number of pixels making up a line
max_line_gap = 20    # maximum gap in pixels between connectable line segments





def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[0, 0, 255], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """

    
    # Find slopes of all lines
    # But only care about lines where abs(slope) > slope_threshold
    left=True
    right=True
    
    slopes = []
    new_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]  # line = [[x1, y1, x2, y2]]
        

        slope = (y2 - y1) / (x2 - x1)
            
        # Filter lines based on slope
        if abs(slope) > 0.56:
            slopes.append(slope)
            new_lines.append(line)
        
    lines = new_lines
    
    # Split lines into right_lines and left_lines, representing the right and left lane lines
    # Right/left lane lines must have positive/negative slope, and be on the right/left half of the image
    right_lines = []
    left_lines = []
    for i, line in enumerate(lines):
        x1, y1, x2, y2 = line[0]
        img_x_center = img.shape[1] / 2  # x coordinate of center of image
        if slopes[i] > 0 and x1 > img_x_center and x2 > img_x_center:
            right_lines.append(line)
        elif slopes[i] < 0 and x1 < img_x_center and x2 < img_x_center:
            left_lines.append(line)
            
    # Run linear regression to find best fit line for right and left lane lines
    # Right lane lines
    right_lines_x = []
    right_lines_y = []
    right_curve=[]
    for line in right_lines:
        x1, y1, x2, y2 = line[0]
        
        right_lines_x.append(x1)
        right_lines_x.append(x2)
        
        right_lines_y.append(y1)
        right_lines_y.append(y2)
        right_curve.append([x1,y1])
        right_curve.append([x2,y2])
        
    if len(right_lines_x) > 0:
        right_m, right_b = np.polyfit(right_lines_x, right_lines_y, 1)  # y = m*x + b
    else:
        right=False



        
    # Left lane lines
    left_lines_x = []
    left_lines_y = []
    left_curve=[]
    
    for line in left_lines:
        x1, y1, x2, y2 = line[0]
        
        left_lines_x.append(x1)
        left_lines_x.append(x2)
        
        left_lines_y.append(y1)
        left_lines_y.append(y2)
        left_curve.append([x1,y1])
        left_curve.append([x2,y2])
        
    if len(left_lines_x) > 0:
        left_m, left_b = np.polyfit(left_lines_x, left_lines_y, 1)  # y = m*x + b
    else:
        left=False


        
    
    # Find 2 end points for right and left lines, used for drawing the line
    # y = m*x + b --> x = (y - b)/m
    y1 = img.shape[0]
    y2 = img.shape[0] * (1 - 0.4)
    if(right):
     right_x1 = (y1 - right_b) / right_m
     right_x2 = (y2 - right_b) / right_m
    
    if(left):
     left_x1 = (y1 - left_b) / left_m
     left_x2 = (y2 - left_b) / left_m
    
    # Convert calculated end points from float to int
    y1 = int(y1)
    y2 = int(y2)
    if(right):
     right_x1 = int(right_x1)
     right_x2 = int(right_x2)
    if(left):    
     left_x1 = int(left_x1)
     left_x2 = int(left_x2)
    
    if(right):
     cv2.line(img, (right_x1, y1), (right_x2, y2), color, thickness)
    
    if(left):
     cv2.line(img, (left_x1, y1), (left_x2, y2), color, thickness)
   
           
             
            
      
            
            

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.9, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def pipeline(image):
  imshape = image.shape
  
  vertices =     vertices = np.array([[\
        ((imshape[1] * (0.08)) // 2, imshape[0]),\
        ((imshape[1] * (0.9)) // 2, imshape[0] - imshape[0] * 0.4),\
        (imshape[1] - (imshape[1] * (0.9)) // 2, imshape[0] - imshape[0] * 0.4),\
        (imshape[1] - (imshape[1] * (0.1)) // 2, imshape[0])]]\
        , dtype=np.int32)

  
  gray=grayscale(image)
  
  blur_gray=gaussian_blur(gray, kernel_size)
  
  edges=canny(blur_gray, low_threshold, high_threshold)
  
  roi=region_of_interest(edges,vertices)

  line_img = hough_lines(roi, rho, theta, threshold,
                            min_line_length, max_line_gap)

  weight_image=weighted_img(line_img,image) 
  
  return weight_image



def pipeline_save(image,image_name):
  
  weight_image=pipeline(image)
  
  cv2.imwrite('result_images/'+image_name,weight_image)
  


def process_image(image):
    result = pipeline(image)
    return result



list_images=os.listdir("test_images/")

for image in list_images:
 image_test='test_images/'+image
 image1 = cv2.imread(image_test)
 pipeline_save(image1,image)

print("Image Processing Done Files have been saved at result_images folder")



white_output = 'result_videos/solidWhiteRight.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)




yellow_output = 'result_videos/solidYellowLeft.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)




challenge_output = 'result_videos/extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

Image Processing Done Files have been saved at result_images folder
[MoviePy] >>>> Building video result_videos/solidWhiteRight.mp4
[MoviePy] Writing video result_videos/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:07<00:00, 29.69it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result_videos/solidWhiteRight.mp4 

CPU times: user 3.61 s, sys: 1.1 s, total: 4.7 s
Wall time: 8.46 s
[MoviePy] >>>> Building video result_videos/solidYellowLeft.mp4
[MoviePy] Writing video result_videos/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:23<00:00, 29.07it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result_videos/solidYellowLeft.mp4 

CPU times: user 11.5 s, sys: 3.3 s, total: 14.7 s
Wall time: 24.5 s
[MoviePy] >>>> Building video result_videos/extra.mp4
[MoviePy] Writing video result_videos/extra.mp4


100%|██████████| 251/251 [00:17<00:00, 15.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result_videos/extra.mp4 

CPU times: user 8.6 s, sys: 2.13 s, total: 10.7 s
Wall time: 19.1 s
